In [1]:
import sagemaker
import boto3
from tqdm import tqdm
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
import time
import json

In [3]:
ENDPOINT_NAME = 'nomic-embed-text-v1-5-2024-03-27-16-26-36-endpoint'

In [4]:
def invoke(data):
    client = boto3.client("sagemaker-runtime", region_name='us-east-2')
    try:
        response = client.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            Body=data,
            ContentType='application/json',
        )
    except:
        print("Error in invoke endpoint")
        return 503, False
    status_code = response['ResponseMetadata']['HTTPStatusCode']
    got_embeddings = True
    try:
        resp = json.loads(response["Body"].read().decode())
        arr = np.array(resp["embeddings"], dtype=np.float16)
        assert 
    except Exception as e:
        print("Did not get embeddings")
        print(e)
        got_embeddings = False
    return status_code, got_embeddings

In [5]:
def generate_sample(batch_size=32, num_tokens=2048):
    # 'else' is one token word, all spaces count as one token
    sentence = " ".join(["else"] * (num_tokens - 1))
    return [sentence] * batch_size

In [10]:
num_tokens = 8192
tok_results = []
batch_size = [8, 16, 32]
for batch in tqdm(batch_size):
    sample = generate_sample(batch, num_tokens)
    data = json.dumps({"texts": sample})
    start = time.time()
    status_code, got_embeddings = invoke(data)
    end = time.time()
    tok_results.append((batch, end - start, status_code, got_embeddings))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.57s/it]


In [11]:
tok_results

[(8, 2.2696588039398193, 200, True),
 (16, 3.8574917316436768, 200, True),
 (32, 7.5540032386779785, 200, True)]